In [1]:
using ITensors
import KrylovKit 
#using KrylovKit

We start by defining the open quantum system Hamiltonian.
$$H = H_S +H_E + H_{SE}$$
with 
$$H_S = \frac{\epsilon}{2} \sigma_z$$
$$H_E = \int_{\omega_{min}}^{\omega_{max}} \text{d}\omega \ \omega a_\omega^\dagger a_\omega$$
$$H_{SE} = \sigma_x \int_{\omega_{min}}^{\omega_{max}} \text{d}\omega \ \sqrt{J(\omega)}(a_\omega^\dagger +a_\omega)$$

The function $J(\omega)$ is the spectral density, determining the interaction strength between the system and the envirnomental modes.

Here and in what follows we will assume the system to be in the a factorized pure initial state $\ket{\psi(0)}_S \otimes \ket{0}_E$, where $\ket{0}_E$ is the vacuum state of the environment, i.e. with each of the environmental modes in the vacuum.


The environment is unitarily reshaped into an equivalent chain of transformed modes $b_j^{(\dagger)}$ satisfying the canonical commutation relation $[b_i,b_j^\dagger]=\delta_{i,j}$ with nearest-neighbor interaction. The system interacts only with the first site of the chain. $H_E$ and $H_{SE}$ become

$$H_E = \sum_{j=1}^{N-1} \kappa_j (b_{j+1}^\dagger b_j + b_{j}^\dagger b_{j+1}) + \sum_{j=1}^N \omega_j b_j^\dagger b_j $$
$$H_{SE} = \kappa_0 \sigma_x (b_j^\dagger+b_j)$$

The coefficients $\kappa_j$ and $\omega_j$ are derived using orthogonal polynomials.

The unitary identity between the original and the transformed environment Hamiltonian $H_E$ holding only in the limit $N \to \infty$. In practice, $N < \infty$, and the value of $N$ is set as to have practical equivalence up to a simulation time $t_\text{max} = f(N)$. 

We load such coefficients from files, which have been produced somewhere else (old, trusty, PyOrthpol). 

In [2]:
using DelimitedFiles
coups = readdlm("ohmic_1.0_c_1.0_T_77.0_coups.dat");
freqs = readdlm("ohmic_1.0_c_1.0_T_77.0_freqs.dat");
#system energy gap
eps = 1.

1.0

In [3]:
sys = siteinds("S=1/2",1);
NChain = 100;
env = siteinds("Boson", dim=5, NChain);
NN = NChain + 1;
sysenv = vcat(sys,env);

In [4]:
stateSys = ["Up"];
stateEnv = ["0" for n=1:NChain];
stateSE = vcat(stateSys,stateEnv);
psi0 = productMPS(sysenv,stateSE);

In [5]:
ampo = OpSum()
#system Hamiltonian
ampo += eps,"Sz",1;
#system-env interaction
#!Sx = 0.5 σx
ampo += 2*coups[1],"Sx",1,"Adag",2
ampo += 2*coups[1],"Sx",1,"A",2
#chain local Hamiltonians
for j=2:NChain
   ampo += freqs[j-1],"N",j
end

for j=2:NChain-1
   ampo += coups[j],"A",j,"Adag",j+1
   ampo += coups[j],"Adag",j,"A",j+1
end
H = MPO(ampo,sysenv); 

We have created the MPO for the open system+chain system.

Now we need to turn to the most difficult part, namely the implementation of the TDVP algorithm. While it would be best to start by the single site version, we exploit Lucas notes to try an implementation of the two-site version. This will be a pain in the ars, but here we are....

## Fundamental steps

1. Prepare the structure to efficiently compute the effective Hamitonians

In [6]:
#initialize the structure needed to efficiently compute effective Hamiltonians
PH = ProjMPO(H);
#Some info...
#PH.nsite,  PH.lpos,  PH.rpos

(2, 0, 102)

2. Set the orthogonality center of the MPS to the first site.

In [18]:
#Set the orthogonality center to site 1 
orthogonalize!(psi0, 1);
#compute the first 2-site tensor A[1]
phi = psi0[1]*psi0[2]

ITensor ord=3 (dim=2|id=387|"S=1/2,Site,n=1") (dim=5|id=529|"Boson,Site,n=1") (dim=1|id=428|"Link,l=2")
ITensors.NDTensors.Dense{Float64, Vector{Float64}}

3. Compute the effective Hamiltonian for leaving the first two tensors out of the projection

In [9]:

#Compute the effective Hamiltonian for the first two sites.
position!(PH,psi0,1);

4. Apply the exponential of the effective Hamiltonian computed at step 3 to T[1] = A[1]*A[2]

In [16]:

# vals, vecs = eigsolve(
#             PH,
#             phi,
#             1
#             );
#Here the magic. All the methond in KrylovKit require the linear map to be
#explicitly defined (matrix), or implicitly defined.
#The overloading of the () operator has done the trick....
KrylovKit.exponentiate(PH,0.1,phi,tol=1e-12)

(ITensor ord=3
Dim 1: (dim=2|id=387|"S=1/2,Site,n=1")
Dim 2: (dim=5|id=529|"Boson,Site,n=1")
Dim 3: (dim=1|id=428|"Link,l=2")
ITensors.NDTensors.Dense{Float64, Vector{Float64}}
 2×5×1
[:, :, 1] =
 1.240187265414691  0.0                …  0.026894099232780962
 0.0                0.693086435557106     0.0, ConvergenceInfo: one converged value after 1 iterations and 7 applications of the linear map;
norms of residuals are given by (1.909599011510105e-37,).
)

## Otherwise...god bless Ori!

In [21]:
using TimeEvoMPS

In [43]:
sysenv[1:1]

1-element Vector{Index{Int64}}:
 (dim=2|id=387|"S=1/2,Site,n=1")

In [48]:
cb = LocalMeasurementCallback(["N"], sysenv,0.01)

LocalMeasurementCallback
Operators: ["N"]
No measurements performed


In [49]:
cb

LocalMeasurementCallback
Operators: ["N"]
No measurements performed


In [50]:
tdvp!(psi0,H,0.01,0.5,maxdim=20,callback=cb)

LoadError: ArgumentError: Overload of "op" or "op!" functions not found for operator name "N" and Index tags: "S=1/2,Site,n=1")

In [38]:
psi0;

In [32]:
psi = psi0;
orthogonalize!(psi,1);
psidag = dag(prime(psi[1],"Site"));
scalar(psidag*op(sysenv,"Sz",1)*psi[1])

0.05065687462525856 + 0.0im

In [37]:
using PyPlot
ts = measurement_ts(cb)
for o in ["x","y","z"]
    S5 = getindex.(measurements(cb)["S$o"],1)
    plot(ts,S5,"-o",label="\$S^$(o)_5\$")
end
legend()
xlabel("t")

1-element Vector{Index{Int64}}:
 (dim=2|id=387|"S=1/2,Site,n=1")